In [7]:
import asyncio
import json
import random
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional, Any
from enum import Enum
import google.generativeai as genai

In [8]:
API_KEY = None

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [9]:
class AgentType(Enum):
    ANALYZER = "analyzer"
    RESEARCHER = "researcher"
    SYNTHESIZER = "synthesizer"
    VALIDATOR = "validator"

@dataclass
class Message:
    sender: str
    receiver: str
    content: str
    msg_type: str
    metadata: Dict = None

In [10]:
class GeminiAgent:
    def __init__(self, agent_id: str, agent_type: AgentType, network: 'AgentNetwork'):
        self.id = agent_id
        self.type = agent_type
        self.network = network
        self.model = genai.GenerativeModel('gemini-2.0-flash')
        self.inbox = asyncio.Queue()
        self.context_memory = []

        self.system_prompts = {
            AgentType.ANALYZER: "You are a data analyzer. Break down complex problems into components and identify key patterns.",
            AgentType.RESEARCHER: "You are a researcher. Gather information and provide detailed context on topics.",
            AgentType.SYNTHESIZER: "You are a synthesizer. Combine information from multiple sources into coherent insights.",
            AgentType.VALIDATOR: "You are a validator. Check accuracy and consistency of information and conclusions."
        }

    async def process_message(self, message: Message):
        """Process incoming message and generate response"""
        if not API_KEY:
            return "❌ API key not configured. Please set API_KEY variable."

        prompt = f"""
        {self.system_prompts[self.type]}

        Context from previous interactions: {json.dumps(self.context_memory[-3:], indent=2)}

        Message from {message.sender}: {message.content}

        Provide a focused response (max 100 words) that adds value to the network discussion.
        """

        try:
            response = await asyncio.to_thread(
                self.model.generate_content, prompt
            )
            return response.text.strip()
        except Exception as e:
            return f"Error processing: {str(e)}"

    async def send_message(self, receiver_id: str, content: str, msg_type: str = "task"):
        """Send message to another agent"""
        message = Message(self.id, receiver_id, content, msg_type)
        await self.network.route_message(message)

    async def broadcast(self, content: str, exclude_self: bool = True):
        """Broadcast message to all agents in network"""
        for agent_id in self.network.agents:
            if exclude_self and agent_id == self.id:
                continue
            await self.send_message(agent_id, content, "broadcast")

    async def run(self):
        """Main agent loop"""
        while True:
            try:
                message = await asyncio.wait_for(self.inbox.get(), timeout=1.0)

                response = await self.process_message(message)

                self.context_memory.append({
                    "from": message.sender,
                    "content": message.content,
                    "my_response": response
                })

                if len(self.context_memory) > 10:
                    self.context_memory = self.context_memory[-10:]

                print(f"🤖 {self.id} ({self.type.value}): {response}")

                if random.random() < 0.3:
                    other_agents = [aid for aid in self.network.agents.keys() if aid != self.id]
                    if other_agents:
                        target = random.choice(other_agents)
                        await self.send_message(target, f"Building on that: {response[:50]}...")

            except asyncio.TimeoutError:
                continue
            except Exception as e:
                print(f"❌ Error in {self.id}: {e}")

In [11]:
class AgentNetwork:
    def __init__(self):
        self.agents: Dict[str, GeminiAgent] = {}
        self.message_log = []
        self.running = False

    def add_agent(self, agent_type: AgentType, agent_id: Optional[str] = None):
        """Add new agent to network"""
        if not agent_id:
            agent_id = f"{agent_type.value}_{len(self.agents)+1}"

        agent = GeminiAgent(agent_id, agent_type, self)
        self.agents[agent_id] = agent
        print(f"✅ Added {agent_id} to network")
        return agent_id

    async def route_message(self, message: Message):
        """Route message to target agent"""
        self.message_log.append(asdict(message))

        if message.receiver in self.agents:
            await self.agents[message.receiver].inbox.put(message)
        else:
            print(f"⚠️  Agent {message.receiver} not found")

    async def initiate_task(self, task: str):
        """Start a collaborative task"""
        print(f"🚀 Starting task: {task}")

        analyzer_agents = [aid for aid, agent in self.agents.items()
                          if agent.type == AgentType.ANALYZER]

        if analyzer_agents:
            initial_message = Message("system", analyzer_agents[0], task, "task")
            await self.route_message(initial_message)

    async def run_network(self, duration: int = 30):
        """Run the agent network for specified duration"""
        self.running = True
        print(f"🌐 Starting agent network for {duration} seconds...")

        agent_tasks = [agent.run() for agent in self.agents.values()]

        try:
            await asyncio.wait_for(asyncio.gather(*agent_tasks), timeout=duration)
        except asyncio.TimeoutError:
            print("⏰ Network session completed")
        finally:
            self.running = False

In [12]:
async def demo_agent_network():
    """Demonstrate the Gemini Agent Network Protocol"""

    network = AgentNetwork()

    network.add_agent(AgentType.ANALYZER, "deep_analyzer")
    network.add_agent(AgentType.RESEARCHER, "info_gatherer")
    network.add_agent(AgentType.SYNTHESIZER, "insight_maker")
    network.add_agent(AgentType.VALIDATOR, "fact_checker")

    task = "Analyze the potential impact of quantum computing on cybersecurity"

    network_task = asyncio.create_task(network.run_network(20))
    await asyncio.sleep(1)
    await network.initiate_task(task)
    await network_task

    print(f"\n📊 Network completed with {len(network.message_log)} messages exchanged")
    agent_participation = {aid: sum(1 for msg in network.message_log if msg['sender'] == aid)
                          for aid in network.agents}
    print("Agent participation:", agent_participation)

def setup_api_key():
    """Interactive API key setup"""
    global API_KEY

    if IN_COLAB:
        from google.colab import userdata
        try:
            API_KEY = userdata.get('GEMINI_API_KEY')
            genai.configure(api_key=API_KEY)
            print("✅ API key loaded from Colab secrets")
            return True
        except:
            print("💡 To use Colab secrets: Add 'GEMINI_API_KEY' in the secrets panel")

    print("🔑 Please enter your Gemini API key:")
    print("   Get it from: https://makersuite.google.com/app/apikey")

    try:
        if IN_COLAB:
            from google.colab import userdata
            API_KEY = input("Paste your API key here: ").strip()
        else:
            import getpass
            API_KEY = getpass.getpass("Paste your API key here: ").strip()

        if API_KEY and len(API_KEY) > 10:
            genai.configure(api_key=API_KEY)
            print("✅ API key configured successfully!")
            return True
        else:
            print("❌ Invalid API key")
            return False
    except KeyboardInterrupt:
        print("\n❌ Setup cancelled")
        return False

In [6]:
if __name__ == "__main__":
    print("🧠 Gemini Agent Network Protocol")
    print("=" * 40)

    if not setup_api_key():
        print("❌ Cannot run without valid API key")
        exit()

    print("\n🚀 Starting demo...")

    if IN_COLAB:
        import nest_asyncio
        nest_asyncio.apply()
        loop = asyncio.get_event_loop()
        loop.run_until_complete(demo_agent_network())
    else:
        asyncio.run(demo_agent_network())

🧠 Gemini Agent Network Protocol
💡 To use Colab secrets: Add 'GEMINI_API_KEY' in the secrets panel
🔑 Please enter your Gemini API key:
   Get it from: https://makersuite.google.com/app/apikey
Paste your API key here: ENTER YOUR KEY
✅ API key configured successfully!

🚀 Starting demo...
✅ Added deep_analyzer to network
✅ Added info_gatherer to network
✅ Added insight_maker to network
✅ Added fact_checker to network
🌐 Starting agent network for 20 seconds...
🚀 Starting task: Analyze the potential impact of quantum computing on cybersecurity
🤖 deep_analyzer (analyzer): Quantum computing poses a dual threat and opportunity for cybersecurity.

**Threat:** Existing encryption methods (RSA, ECC) become vulnerable to Shor's algorithm, potentially exposing sensitive data.

**Opportunity:** Quantum Key Distribution (QKD) and post-quantum cryptography (PQC) offer potentially unbreakable encryption, securing future communications.

Impact necessitates a proactive shift: investing in PQC research, d